In [ ]:
    import os
    os.environ['PATH'] = os.environ['PATH'] + ':/usr/local/cuda/bin'

    import sys
    print sys.getrecursionlimit()
    sys.setrecursionlimit(40000)
    print sys.getrecursionlimit()
    
    import theano
    import theano.tensor as T
    import theano.tensor.signal.conv
    import numpy as np
    import scipy.io as sio
    import time

    from lasagne.layers import InputLayer
    from lasagne.layers import DenseLayer
    from lasagne.layers import NonlinearityLayer
    from lasagne.layers import DropoutLayer
    from lasagne.layers import Pool2DLayer
    from lasagne.layers import Conv2DLayer as ConvLayer
    import lasagne.layers as ll
    
    import numpy as np

    ###############################
    n_filt_grad = 8;

    filt_grad = np.zeros((n_filt_grad,1,3,3), dtype=np.float32)
    bias_grad = np.zeros((n_filt_grad,), dtype=np.float32)

    filt_grad[0,0,:,:] = [[0, 0, 0],
                          [0, -1,1],
                          [0, 0, 0]];

    filt_grad[1,0,:,:] = [[0, 0, 0],
                          [0, -1, 0],
                          [0, 1, 0]];
    filt_grad[2,0,:,:] = [[0, 0, 0],
                          [0, -1,1],
                          [0, 0, 0]];

    filt_grad[3,0,:,:] = [[0, 0, 0],
                          [0, -1, 0],
                          [0, 1, 0]]; 

    filt_grad[4:8,:,:] = filt_grad[0:4,:,:]
    
    n_filt_div = 1;

    filt_div = np.zeros((1,n_filt_grad,3,3), dtype=np.float32)
    bias_div = np.zeros((1,), dtype=np.float32)

    filt_div[0,0,:,:] =  [[0, 0, 0],
                          [-1, 1, 0],
                          [0, 0, 0]];

    filt_div[0,1,:,:] =  [[0, -1, 0],
                          [0, 1, 0],
                          [0, 0, 0]]; 
    
    filt_div[0,2,:,:] =  [[0, 0, 0],
                          [-1, 1, 0],
                          [0, 0, 0]];

    filt_div[0,3,:,:] =  [[0, -1, 0],
                          [0, 1, 0],
                          [0, 0, 0]];
    
    filt_div = filt_div * float(2.0) / float(n_filt_grad);
    
    filt_div[0,4:8,:,:] = filt_div[0,0:4,:,:]
    
    print filt_div
    print filt_grad
    ###############################

    def interpolate_varsize(im, x, y, out_height, out_width):
        # *_f are floats
        num_batch, height, width, channels = im.shape
        height_f = T.cast(height, theano.config.floatX)
        width_f = T.cast(width, theano.config.floatX)

        n = height * width * num_batch;
        
        x = T.clip(x, 0, width_f - 1)
        y = T.clip(y, 0, height_f - 1)

        # obtain indices of the 2x2 pixel neighborhood surrounding the coordinates;
        # we need those in floatX for interpolation and in int64 for indexing. for
        # indexing, we need to take care they do not extend past the image.
        x0_f = T.floor(x)
        y0_f = T.floor(y)
        x1_f = x0_f + 1.0
        y1_f = y0_f + 1.0
        x0 = T.cast(x0_f, 'int64')
        y0 = T.cast(y0_f, 'int64')
        x1 = T.cast(T.minimum(x1_f, width_f - 1), 'int64')
        y1 = T.cast(T.minimum(y1_f, height_f - 1), 'int64')
        
        # The input is [num_batch, height, width, channels]. We do the lookup in
        # the flattened input, i.e [num_batch*height*width, channels]. We need
        # to offset all indices to match the flat version
        dim2 = width
        dim1 = width*height
        base = T.repeat(
            T.arange(num_batch, dtype='int64')*dim1, out_height*out_width)
        
        base_y0 = base + y0*dim2
        base_y1 = base + y1*dim2
        idx_a = base_y0 + x0
        idx_b = base_y1 + x0
        idx_c = base_y0 + x1
        idx_d = base_y1 + x1

        # use indices to lookup pixels for all samples
        im_flat = im.reshape((-1, channels))
        
        Ia = im_flat[idx_a]
        Ib = im_flat[idx_b]
        Ic = im_flat[idx_c]
        Id = im_flat[idx_d]

        # calculate interpolated values
        wa = ((x1_f-x) * (y1_f-y)).dimshuffle(0, 'x')
        wb = ((x1_f-x) * (y-y0_f)).dimshuffle(0, 'x')
        wc = ((x-x0_f) * (y1_f-y)).dimshuffle(0, 'x')
        wd = ((x-x0_f) * (y-y0_f)).dimshuffle(0, 'x')
        output = T.sum([wa*Ia, wb*Ib, wc*Ic, wd*Id], axis=0)
        
        return output
    
    def interpolate_varsize_nn(im, x, y, out_height, out_width):
        # *_f are floats
        num_batch, height, width, channels = im.shape
        height_f = T.cast(height, theano.config.floatX)
        width_f = T.cast(width, theano.config.floatX)

        n = height * width * num_batch;
        
        x = T.clip(x, 0, width_f - 1)
        y = T.clip(y, 0, height_f - 1)

        # obtain indices of the 2x2 pixel neighborhood surrounding the coordinates;
        # we need those in floatX for interpolation and in int64 for indexing. for
        # indexing, we need to take care they do not extend past the image.
        
        x0_f = T.round(x)
        y0_f = T.round(y)
        
        x0 = T.cast(x0_f, 'int64')
        y0 = T.cast(y0_f, 'int64')
        
        # The input is [num_batch, height, width, channels]. We do the lookup in
        # the flattened input, i.e [num_batch*height*width, channels]. We need
        # to offset all indices to match the flat version
        dim2 = width
        dim1 = width*height
        base = T.repeat(
            T.arange(num_batch, dtype='int64')*dim1, out_height*out_width)
        
        base_y0 = base + y0*dim2
        idx = base_y0 + x0

        # use indices to lookup pixels for all samples
        im_flat = im.reshape((-1, channels))
       
        I = im_flat[idx]
        
        return I
    
    def _meshgrid(height, width):
        x_t = T.dot(T.ones((height, 1),dtype='float32'),
                T.arange(width, dtype='float32').dimshuffle('x', 0))
        y_t = T.dot(T.arange(height, dtype='float32').dimshuffle(0, 'x'),
                T.ones((1, width),dtype='float32'))

        x_t_flat = x_t.reshape((1, -1))
        y_t_flat = y_t.reshape((1, -1))

        return (x_t_flat,y_t_flat)
    
    def _meshgrid_nonflat(height, width):
        x_t = T.dot(T.ones((height, 1),dtype='float32'),
                T.arange(width, dtype='float32').dimshuffle('x', 0))
        y_t = T.dot(T.arange(height, dtype='float32').dimshuffle(0, 'x'),
                T.ones((1, width),dtype='float32'))

        return (x_t,y_t)

    def warp(imgs, u, v):
        num_batch, channels, height, width = imgs.shape
        
        [x_grid,y_grid] = _meshgrid_nonflat(height, width);
        
        x_grid_batch = T.tile(x_grid, (n_batch,1,1,1))
        y_grid_batch = T.tile(y_grid, (n_batch,1,1,1))
        
        warped_grid_x = x_grid_batch + u;
        warped_grid_y = y_grid_batch + v;

        num_batch, num_channels, height, width = imgs.shape
        # dimshuffle input to  (bs, height, width, channels)
        input_dim = imgs.dimshuffle(0, 2, 3, 1)
        input_grid_x = warped_grid_x.dimshuffle(0, 2, 3, 1).flatten();
        input_grid_y = warped_grid_y.dimshuffle(0, 2, 3, 1).flatten();
        
        height_int = T.cast(height, 'int64')
        width_int = T.cast(width, 'int64')
        input_transformed = interpolate_varsize(input_dim, input_grid_x, input_grid_y, height_int, width_int)

        output = T.reshape(input_transformed, (num_batch, height, width, num_channels))
        output = output.dimshuffle(0, 3, 1, 2)
        return output  
    
    def img_bca_deriv(I1, I2, u, v):
        I2_warped = warp(I2, u, v);
        delta = 0.5 * T.ones_like(u,dtype='float32');
        u_m = u - delta;
        u_p = u + delta;
        v_m = v - delta;
        v_p = v + delta;
        
        I2_x_m = warp(I2, u_m, v);
        I2_y_m = warp(I2, u, v_m);
        I2_x_p = warp(I2, u_p, v);
        I2_y_p = warp(I2, u, v_p);
        
        Ix = I2_x_p - I2_x_m;
        Iy = I2_y_p - I2_y_m;
        
        It = I2_warped - I1;
        
        return(Ix,Iy,It)
    
    def imresize(imgs, height_new, width_new):
        num_batch, num_channels, height, width = imgs.shape
        
        out_height = T.cast(height_new, 'int64')
        out_width = T.cast(width_new, 'int64')
        
        [x_grid,y_grid] = _meshgrid_nonflat(out_height, out_width);
        x_s = T.tile(x_grid, (n_batch,1,1,1))
        y_s = T.tile(y_grid, (n_batch,1,1,1))
        
        height_f = T.cast(height, dtype='float32')
        width_f = T.cast(width, dtype='float32')
        
        height_new_f = T.cast(out_height, dtype='float32')
        width_new_f = T.cast(out_width, dtype='float32')
        
        x_s_flat = x_s.flatten() * (width_f-1.0) /  (width_new_f-1.0)
        y_s_flat = y_s.flatten() * (height_f-1.0) / (height_new_f-1.0)
        
        input_dim = imgs.dimshuffle(0, 2, 3, 1)
        input_transformed = interpolate_varsize(input_dim, x_s_flat, y_s_flat, out_height, out_width)
                
        output = T.reshape(input_transformed, (num_batch, out_height, out_width, num_channels))
        output = output.dimshuffle(0, 3, 1, 2)  # dimshuffle to conv format

        return output
    
    def imresize_nn(imgs, height_new, width_new):
        num_batch, num_channels, height, width = imgs.shape
        
        out_height = T.cast(height_new, 'int64')
        out_width = T.cast(width_new, 'int64')
        
        [x_grid,y_grid] = _meshgrid_nonflat(out_height, out_width);
        x_s = T.tile(x_grid, (n_batch,1,1,1))
        y_s = T.tile(y_grid, (n_batch,1,1,1))
        
        height_f = T.cast(height, theano.config.floatX)
        width_f = T.cast(width, theano.config.floatX)
        
        height_new_f = T.cast(out_height, theano.config.floatX)
        width_new_f = T.cast(out_width, theano.config.floatX)
        
        x_s_flat = x_s.flatten() * (width_f-1.0) /  (width_new_f-1.0)
        y_s_flat = y_s.flatten() * (height_f-1.0) / (height_new_f-1.0)
        
        input_dim = imgs.dimshuffle(0, 2, 3, 1)
        input_transformed = interpolate_varsize_nn(input_dim, x_s_flat, y_s_flat, out_height, out_width)
                
        output = T.reshape(input_transformed, (num_batch, out_height, out_width, num_channels))
        output = output.dimshuffle(0, 3, 1, 2)  # dimshuffle to conv format

        return output
    
    def data_term_bca_proximal(u, v, u0, v0, I_x, I_y, I_t, theta, lmbd, beta):
        I_grad_sqr = T.maximum(1e-09, I_x*I_x + I_y*I_y + beta*beta);    
            
        rho = I_t + (u - u0) * I_x + (v - v0) * I_y;
        
        threshold = theta * lmbd * I_grad_sqr;
        
        mask1 = (rho < -1.0 * threshold);
        mask2 = (rho > threshold);
        mask3 = (abs(rho) < threshold);

        u = u + theta * lmbd * I_x * mask1; 
        v = v + theta * lmbd * I_y * mask1;
        
        u = u - theta * lmbd * I_x * mask2; 
        v = v - theta * lmbd * I_y * mask2;
        
        u = u - (rho * I_x / I_grad_sqr) * mask3; 
        v = v - (rho * I_y / I_grad_sqr) * mask3;
        
        return (u,v)
        
    def reproject_dual(dual_u, dual_v, beta):
        norm_sqr = T.sqr(dual_v[:,0,:,:]) + T.sqr(dual_v[:,1,:,:]) + T.sqr(dual_u[:,2,:,:]) + T.sqr(dual_u[:,3,:,:]) + T.sqr(dual_v[:,4,:,:]) + T.sqr(dual_v[:,5,:,:]) + T.sqr(dual_u[:,6,:,:]) + T.sqr(dual_u[:,7,:,:]); 
        
        reprojection = T.maximum(T.sqrt(norm_sqr+beta*beta),1.0)
        
        dual_u_unit = dual_u / reprojection[:,np.newaxis,:,:];
        dual_v_unit = dual_v / reprojection[:,np.newaxis,:,:];
                
        return (dual_u_unit,dual_v_unit)
    
    def dual_update(u, v, dual_u, dual_v, theta, lipschitz_c, beta):
        grad_u = ConvLayer(InputLayer(shape=(None, 1, None, None), input_var=u), 
                           name = "l_conv1_1", num_filters=n_filt_grad, filter_size=3, pad=1,
                           nonlinearity=None, flip_filters=False);
        
        grad_v = ConvLayer(InputLayer(shape=(None, 1, None, None), input_var=v), 
                           name = "l_conv1_1", num_filters=n_filt_grad, filter_size=3, pad=1,
                           nonlinearity=None, flip_filters=False);

        grad_u.W.set_value(filt_grad)
        grad_u.b.set_value(bias_grad)
        
        grad_v.W.set_value(filt_grad)
        grad_v.b.set_value(bias_grad)
                
        dual_u_new = dual_u + 1 / lipschitz_c * theta * ll.get_output(grad_u);
        dual_v_new = dual_v + 1 / lipschitz_c * theta * ll.get_output(grad_v);
            
        [dual_u_new, dual_v_new] = reproject_dual(dual_u_new, dual_v_new, beta);
        
        return (dual_u_new, dual_v_new, grad_u, grad_v)
    
    def primal_update(u, v, dual_u, dual_v, theta):
        div_dual_u = ConvLayer(InputLayer(shape=(None, n_filt_grad, None, None), input_var=dual_u), 
                               name = "l_conv1_2", num_filters=n_filt_div, filter_size=3, pad=1,
                               nonlinearity=None, flip_filters=False);
        
        div_dual_v = ConvLayer(InputLayer(shape=(None, n_filt_grad, None, None), input_var=dual_v), 
                               name = "l_conv1_2", num_filters=n_filt_div, filter_size=3, pad=1,
                               nonlinearity=None, flip_filters=False);

        div_dual_u.W.set_value(filt_div)
        div_dual_u.b.set_value(bias_div)
        
        div_dual_v.W.set_value(filt_div)
        div_dual_v.b.set_value(bias_div)
        
        u_new = u + theta * ll.get_output(div_dual_u);
        v_new = v + theta * ll.get_output(div_dual_v);
        
        return (u_new, v_new, div_dual_u, div_dual_v);


In [ ]:
import os
os.environ['PATH'] = os.environ['PATH'] + ':/usr/local/cuda/bin'

import theano
import theano.tensor as T
import theano.tensor.signal.conv
import numpy as np
import scipy.io as sio
import time

from lasagne.layers import InputLayer
from lasagne.layers import Conv2DLayer as ConvLayer

from lasagne.init import Constant

import lasagne.layers as ll
from lasagne.layers import ConcatLayer
import lasagne.updates

h = 436;
w = 1024;

ratio = 0.8

n_scales = 15;

h_scales = [];
w_scales = [];

h_scale = h;
w_scale = w;

for i_scale in range(0, n_scales):
    h_scales = [h_scale] + h_scales;
    w_scales = [w_scale] + w_scales;

    h_scale = int(h_scale * ratio);
    w_scale = int(w_scale * ratio);

print 'image pyramid:'    
    
print h_scales
print w_scales

n_batch = 4;
n_inner_iter = 10;

print '-----------------'

tic = time.time()

params_lasagne = []

theta = T.scalar(dtype='float32');
lmbd = T.scalar(dtype='float32');
beta = T.scalar(dtype='float32');
lipschitz_c = T.scalar(dtype='float32');

lr = T.scalar(dtype='float32');

I1_batch = T.tensor4(dtype='float32');
I2_batch = T.tensor4(dtype='float32');

I1_batch_scale_lst = [];
I2_batch_scale_lst = [];

for i_scale in range(0, n_scales-1):
    I1_batch_coarse = imresize(I1_batch, h_scales[i_scale], w_scales[i_scale]);
    I2_batch_coarse = imresize(I2_batch, h_scales[i_scale], w_scales[i_scale]);
    
    I1_batch_scale_lst.append(I1_batch_coarse);
    I2_batch_scale_lst.append(I2_batch_coarse);

I1_batch_scale_lst = I1_batch_scale_lst + [I1_batch];  
I2_batch_scale_lst = I2_batch_scale_lst + [I2_batch];      
    
u_gt = T.tensor4(dtype='float32');
v_gt = T.tensor4(dtype='float32');

lr = T.scalar(dtype='float32');

u_init = T.zeros_like(I1_batch_scale_lst[0],dtype='float32');
v_init = T.zeros_like(I1_batch_scale_lst[0],dtype='float32');

dual_u_init = T.tile(T.zeros_like(I1_batch_scale_lst[0],dtype='float32'),(1,n_filt_grad,1,1))
dual_v_init = T.tile(T.zeros_like(I1_batch_scale_lst[0],dtype='float32'),(1,n_filt_grad,1,1))

################## SCALE ITERATION ######################
for i_scale in range(0,n_scales):
    print '-----------------'
    print 'scale ', i_scale
    
    if (i_scale == 0):
        print 'initialize with zeros'
        u = u_init;
        v = v_init;
        
        dual_u = dual_u_init;
        dual_v = dual_v_init;
    else:    
        rescale_ratio_h = float(h_scales[i_scale]) / float(h_scales[i_scale-1]); 
        rescale_ratio_w = float(w_scales[i_scale]) / float(w_scales[i_scale-1]);
        
        u = u * rescale_ratio_w;
        v = v * rescale_ratio_h;
        
        print 'rescale, ratio = ', rescale_ratio_h

    u0 = u;
    v0 = v;     
        
    [Ix,Iy,It] = img_bca_deriv(I1_batch_scale_lst[i_scale], I2_batch_scale_lst[i_scale], u0, v0);

    for it in range(0,n_inner_iter):
        [u,v] = data_term_bca_proximal(u, v, u0, v0, Ix, Iy, It, theta, lmbd, beta);
        
        [dual_u, dual_v, div_u_layer, div_v_layer] = dual_update(u, v, dual_u, dual_v, theta, lipschitz_c, beta);
         
        [u, v, grad_u_layer, grad_v_layer] = primal_update(u, v, dual_u, dual_v, theta);
        
        #[u,v] = data_term_bca_proximal(u, v, u0, v0, Ix, Iy, It, theta, lmbd, beta);
        
        #u_ = 2.0 * u - u_;
        #v_ = 2.0 * v - v_;
    
        params_lasagne += ll.get_all_params(grad_u_layer) + ll.get_all_params(grad_v_layer) + ll.get_all_params(div_u_layer) + ll.get_all_params(div_v_layer);
    ##############
        
    # rescale    
    if (i_scale <  n_scales - 1):  
        u = imresize_nn(u, h_scales[i_scale+1], w_scales[i_scale+1]);
        v = imresize_nn(v, h_scales[i_scale+1], w_scales[i_scale+1]);
        dual_v = imresize_nn(dual_v, h_scales[i_scale+1], w_scales[i_scale+1]);
        dual_u = imresize_nn(dual_u, h_scales[i_scale+1], w_scales[i_scale+1]);

################## END SCALE ITERATION #####################

theano_optical_flow_fn = theano.function(inputs=[I1_batch,I2_batch,theta,lmbd,beta,lipschitz_c], outputs=[u,v], on_unused_input='ignore');

diff = T.sqrt(T.sqr(u-u_gt) + T.sqr(v-v_gt) + 1.0E-8);

L = T.mean(diff.flatten());

theano_loss_fn = theano.function(inputs=[I1_batch,I2_batch,u_gt,v_gt,theta,lmbd,beta,lipschitz_c], outputs=[L,u,v], on_unused_input='ignore');

updates = lasagne.updates.adam(L, params_lasagne, learning_rate=lr);

theano_train_fn = theano.function(inputs=[I1_batch,I2_batch,u_gt,v_gt,theta,lmbd,beta,lipschitz_c,lr], 
                                  outputs=[L], updates = updates,on_unused_input='ignore');
print 'parameters total:'
print len(params_lasagne)

toc = time.time()

print 'time:'
print toc - tic
